<a href="https://colab.research.google.com/github/danielsoo/AI/blob/main/Fine_Tuning_LLMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install --upgrade tensorflow jax jaxlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 82.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.33
    Uninstalling jaxlib-0.4.33:
      Successfully uninstalled jaxlib-0.4.33
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: jax
    Found existing installation: jax 0.4.33
    Uninstalling jax-0.4.33:
      Successfully uninstalled j

In [1]:
# Import necessary libraries
import os
import zipfile
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator




In [2]:
# Step 1: Download and extract the dataset
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Set paths for the dataset
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-02 00:18:53--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.134.207, 142.251.107.207, 74.125.196.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.134.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   194MB/s    in 0.3s    

2024-11-02 00:18:53 (194 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [3]:
# Step 2: Data preprocessing and augmentation using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values to the range 0-1
    rotation_range=40,          # Range for random rotations
    width_shift_range=0.2,      # Range for random horizontal shifts
    height_shift_range=0.2,     # Range for random vertical shifts
    shear_range=0.2,            # Shear intensity (shear angle in counter-clockwise direction in degrees)
    zoom_range=0.2,             # Range for random zoom
    horizontal_flip=True,       # Randomly flip inputs horizontally
    fill_mode='nearest'         # Fill mode for pixels outside the boundaries
)

validation_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for validation data

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),   # Resize images to match ResNet50 input size
    batch_size=32,
    class_mode='binary'       # Binary classification
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
# Step 3: Load ResNet50 model and add custom layers
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,           # Remove the top layer
    input_shape=(150, 150, 3),   # Specify input shape
    pooling='avg',               # Apply global average pooling
    weights='imagenet'           # Use pretrained ImageNet weights
)

# Add a custom output layer
x = pretrained_model.output
x = Dense(1, activation='sigmoid')(x)  # Sigmoid activation for binary classification

# Define the final model
model = Model(inputs=pretrained_model.input, outputs=x)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# Step 4: Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
# Step 5: Train the model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 885s 14s/step - accuracy: 0.9836 - loss: 0.0584 - val_accuracy: 0.5120 - val_loss: 0.7174
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 946s 15s/step - accuracy: 0.9750 - loss: 0.0625 - val_accuracy: 0.5230 - val_loss: 0.7899
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 886s 14s/step - accuracy: 0.9754 - loss: 0.0541 - val_accuracy: 0.5020 - val_loss: 1.7710
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 888s 14s/step - accuracy: 0.9897 - loss: 0.0405 - val_accuracy: 0.5010 - val_loss: 2.3240
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 893s 14s/step - accuracy: 0.9811 - loss: 0.0528 - val_accuracy: 0.5150 - val_loss: 1.0243


In [8]:
# Step 6: Evaluate the model and define the prediction function
# Model evaluation
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy}")

# Prediction function
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB").resize((150, 150))
    img_array = np.expand_dims(np.array(img) / 255.0, axis=0)  # Normalize and add batch dimension
    prediction = model.predict(img_array)
    return "Dog" if prediction[0] > 0.5 else "Cat"

# Example prediction
example_path = os.path.join(validation_dir, 'cats', os.listdir(os.path.join(validation_dir, 'cats'))[0])
print("Prediction for example image:", predict_image(example_path))

32/32 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.5390 - loss: 1.0236
Validation Accuracy: 0.5149999856948853
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Prediction for example image: Cat
